### Text sentiment analyser

Sarang Balan & Jean-Phillipe Maltais

Run the following code to install the necessary libraries, including some requirements

In [3]:
!pip install shapely
!pip install sentencepiece
import time
from py_pdf_parser.loaders import load_file
from py_pdf_parser.visualise import visualise



Load the document in question

In [2]:

document = load_file("RBC1.pdf")


In [3]:
set(element.font for element in document.elements)

{'ABCDEE+Calibri,10.0',
 'ABCDEE+Calibri,6.0',
 'ABCDEE+Calibri,8.0',
 'ABCDEE+Calibri,Bold,8.0',
 'AUPMZC+Calibri,7.0',
 'AUPMZC+Calibri,8.0',
 'AUPMZC+Calibri,9.0',
 'ECEHIB+Calibri,10.0',
 'ECEHIB+Calibri,11.0',
 'ECEHIB+Calibri,12.0',
 'ECEHIB+Calibri,5.6',
 'ECEHIB+Calibri,5.9',
 'ECEHIB+Calibri,6.0',
 'ECEHIB+Calibri,7.0',
 'ECEHIB+Calibri,7.1',
 'ECEHIB+Calibri,8.0',
 'ECEHIB+Calibri,9.0',
 'ECEHIC+Calibri-Bold,10.0',
 'ECEHIC+Calibri-Bold,11.0',
 'ECEHIC+Calibri-Bold,14.0',
 'ECEHIC+Calibri-Bold,16.0',
 'ECEHIC+Calibri-Bold,4.6',
 'ECEHIC+Calibri-Bold,7.1',
 'ECEHIC+Calibri-Bold,8.0',
 'ECEHID+Arial-BoldMT,5.3',
 'ECEHID+Arial-BoldMT,6.7',
 'ECEHID+Arial-BoldMT,7.7',
 'ECEHIE+ArialMT,4.9',
 'OVENGC+Calibri-Bold,9.0'}

In [4]:
FONT_MAPPING = {
    r"\w{6}\+Calibri,10.0": "body",
}
document = load_file("RBC1.pdf", font_mapping=FONT_MAPPING, font_mapping_is_regex=True)

In [5]:
#visualise(document,show_info=True)

In [6]:
listoftexts = (
    document.elements.filter_by_font("body")
)

In [7]:
listoftexts[0].text()

'Dunelm Group PLC\nDunelm - recent pullback provides a buying\nopportunity; upgrade to Outperform\nOur view: The Dunelm share price has come off c.20% in the past month\non store closures and rotation out of so-called "Covid beneficiaries." We\nthink  it  provides  a  more  attractive  entry  point  to  buy  a  growing,  well-\nmanaged retailer with a range advantage and which is well placed to make\nfurther enhanced cash returns to shareholders. Upgrade to Outperform.'

In [8]:
listoftexts[1].text()

'Key points:\nSuccessful  transition  to  Digital  and  Click  and  Collect.  Frustratingly  for\nDunelm, following a very strong October, homewares were not included\nin  the  list  of  "essential"  categories  for  retailers  operating  in  England\nin  November.  However,  Dunelm\'s  delivery  to  home  service  (c.15%  of\nsales)  provides  a  partial  offset  and  its  stores  should  reopen  again  in\nDecember,  when  we  expect  to  see  pent-up  demand  coming  through.\nDunelm\'s  enhanced  digital  platform  provides  customers  with  a  better\nwebsite experience and delivery options, along with Click & Collect.'

In [9]:
paragraph_dict = {}

for idx,item in enumerate(listoftexts):
    mystring = item.text().replace('\n', ' ')
    paragraph_dict[idx] = mystring

In [10]:
paragraph_dict

{0: 'Dunelm Group PLC Dunelm - recent pullback provides a buying opportunity; upgrade to Outperform Our view: The Dunelm share price has come off c.20% in the past month on store closures and rotation out of so-called "Covid beneficiaries." We think  it  provides  a  more  attractive  entry  point  to  buy  a  growing,  well- managed retailer with a range advantage and which is well placed to make further enhanced cash returns to shareholders. Upgrade to Outperform.',
 1: 'Key points: Successful  transition  to  Digital  and  Click  and  Collect.  Frustratingly  for Dunelm, following a very strong October, homewares were not included in  the  list  of  "essential"  categories  for  retailers  operating  in  England in  November.  However,  Dunelm\'s  delivery  to  home  service  (c.15%  of sales)  provides  a  partial  offset  and  its  stores  should  reopen  again  in December,  when  we  expect  to  see  pent-up  demand  coming  through. Dunelm\'s  enhanced  digital  platform  provi

### Main analysis section

In [11]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration, TFPegasusForConditionalGeneration

# Let's load the model and the tokenizer 
model_name = "human-centered-summarization/financial-summarization-pegasus"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name) # If you want to use the Tensorflow model 
                                                                    # just replace with TFPegasusForConditionalGeneration




Downloading:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

C:\Users\Sarang\miniconda3\envs\finenv\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Sarang\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Saudi bank to pay a 3.5% premium to Samba share price. Gulf region’s third-largest lender will have total assets of $220 billion


In [18]:
# Some text to summarize here
text_to_summarize = paragraph_dict[5]
# Tokenize our text
# If you want to run the code in Tensorflow, please remember to return the particular tensors as simply as using return_tensors = 'tf'
input_ids = tokenizer(text_to_summarize, return_tensors="pt").input_ids
start = time.time()

# Generate the output (Here, we use beam search but you can also use any other strategy you like)
output = model.generate(
    input_ids, 
    max_length=64, 
    num_beams=3, 
    early_stopping=True
)

# Finally, we can print the generated summary
print(tokenizer.decode(output[0], skip_special_tokens=True))
end = time.time()
print(end - start)


Retailer is set up for continued expansion. LTIP gives CEO and CFO more "skin in the game"
8.245872497558594


In [22]:
start = time.time()
for item in range(10):
    
    # Some text to summarize here
    text_to_summarize = paragraph_dict[item]
    
    input_ids = tokenizer(text_to_summarize, return_tensors="pt").input_ids
    

# Generate the output (Here, we use beam search but you can also use any other strategy you like)
    output = model.generate(
    input_ids, 
    max_length=32, 
    num_beams=5, 
    early_stopping=True
    )

    
    
    print(tokenizer.decode(output[0], skip_special_tokens=True))
end = time.time()
print(end - start)

Shares have come off recently on store closures and rotation out of so-called beneficiaries.
homewares not included in list of "essential" categories.
Retailer has strong balance sheet, low debt, strong track record of returning surplus cash.
Retailer has a significant range advantage over its rivals.
We have trimmed our FY22-23 PBT estimates by 3%, still above consensus.
Retailer is set up for continued expansion. LTIP gives CEO and CFO more "skin in the game"
.
Retailer is well-run, has a significant range advantage and is cash generative.
Valuation now looks more attractive after recent dip.
Retailer could be worth more than £2 billion
99.6954927444458
